In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
assignmentsDataSubmissionsFilesPath = str(input())

In [ ]:
assignmentsDataSubmissionsFiles = glob.glob(assignmentsDataSubmissionsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataSubmissionsData = pd.DataFrame()
for assignmentsDataSubmissionsFile in assignmentsDataSubmissionsFiles:
    assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.append(pd.read_csv(assignmentsDataSubmissionsFile), ignore_index = True)
    
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataSubmissionsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[(assignmentsDataSubmissionsData['metadata_event_time'] >= startDate) & (assignmentsDataSubmissionsData['metadata_event_time'] <= endDate)]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[assignmentsDataSubmissionsData['metadata_context_role'] == 'StudentEnrollment']
assignmentsDataSubmissionsData.drop(['body_role', 'body_asset_name', 'collected_at', 'metadata_context_role', 'body_asset_id'], axis = 1, inplace = True)
assignmentsDataSubmissionsData

In [ ]:
ungradedGradedDiscussionForums = pd.read_csv(str(input()))
ungradedGradedDiscussionForums

In [ ]:
assignmentsDataSubmissionsData['course_topic'] = assignmentsDataSubmissionsData['metadata_context_id'] + "-" + assignmentsDataSubmissionsData['body_discussion_topic_id']
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.merge(ungradedGradedDiscussionForums, on=['course_topic'])
assignmentsDataSubmissionsData.drop(['course_topic'], axis = 1, inplace = True)
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsDataUngraded = assignmentsDataSubmissionsData.loc[assignmentsDataSubmissionsData['is_graded'] == False]
assignmentsDataSubmissionsDataUngraded.drop(['is_graded'], axis = 1, inplace = True)
assignmentsDataSubmissionsDataUngraded

In [ ]:
assignmentsDataSubmissionsDataUngraded['metadata_event_time'] = assignmentsDataSubmissionsDataUngraded['metadata_event_time'].apply(str)
assignmentsDataSubmissionsDataUngraded

In [ ]:
assignmentsDataSubmissionsDataUngraded = assignmentsDataSubmissionsDataUngraded.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].apply(list).reset_index()
assignmentsDataSubmissionsDataUngraded

In [ ]:
assignmentsDataSubmissionsDataUngraded.to_csv("UngradedDiscussionForumsAccesses_Intercession_2022.csv", header = True, index = False)

In [ ]:
assignmentsDataSubmissionsDataGraded = assignmentsDataSubmissionsData.loc[assignmentsDataSubmissionsData['is_graded'] == True]
assignmentsDataSubmissionsDataGraded.drop(['is_graded'], axis = 1, inplace = True)
assignmentsDataSubmissionsDataGraded

In [ ]:
assignmentsDataSubmissionsDataGraded['metadata_event_time'] = assignmentsDataSubmissionsDataGraded['metadata_event_time'].apply(str)
assignmentsDataSubmissionsDataGraded

In [ ]:
assignmentsDataSubmissionsDataGraded = assignmentsDataSubmissionsDataGraded.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].apply(list).reset_index()
assignmentsDataSubmissionsDataGraded

In [ ]:
assignmentsDataSubmissionsDataGraded.to_csv("GradedDiscussionForumsAccesses_Intercession_2022.csv", header = True, index = False)